Drew Lickman\
CSCI 4820-001\
Project #3\
Due: 10/9/24

AI Usage Disclaimer:


# Lexicon-Based Sentiment Analysis using Custom Logistic Regression

## Assignment Requirements:

### Input
---

- Positive words
- Negative words
- IMDb reviews

### Processing
---

- There are two classifiers
	- Custom Logistic Regression
	- sklearn LogisticRegression
- Implement a Python class (CustomLogisticRegression)
	- \__init\__(self, learning_rate, num_iters) method
		- self.learning_rate
		- self.num_iters
		- self.weights = None
		- self.bias = None
	- sigmoid(z)
		- return result
	- fit(X, y)
		- Sets weights to correct shape and initializes them to 0
		- Applies batch gradient descent to the entire dataset in a loop for num_iters
	- predict(X)
		- z = w dot x + b
		- return sigmoid(z) 

### Output
---

- For each trial and for each classifier
	- Print the sklearn confusion_matrix and classification_Report
- Output the average of the confusion matrices across trials for each classifier

## Python Code

Markdown above each cell used to explain each block of code

1. Load and preprocess IMDb reviews

In [70]:
import numpy as np
import sklearn

sentimentWords = {}
posWords = {}
negWords = {}
with open("positive-words.txt", encoding="utf-8") as positivewords:
	lines = positivewords.readlines()
	for line in lines:
		if line[0] != ";" and line.strip() != '': 
			posWords[line.rstrip('\n')] = 1
with open("negative-words.txt", encoding="utf-8") as negativewords:
	lines = negativewords.readlines()
	for line in lines:
		if line[0] != ";" and line.strip() != '':
			negWords[line.rstrip('\n')] = 1
sentimentWords = {**posWords, **negWords} # Combine positive words and negative words into one dictionary
#print(sentimentWords)

# Add each line of the IMDb reviews to the reviews array
reviews = []
trueValues = []
with open("imdb_reviews.txt", encoding="utf-8") as imdbreviews:
	lines = imdbreviews.readlines()
	for line in lines:
		splitLine = line.rstrip().rsplit(' ', 1)
		reviews.append(splitLine[0]) # removes true sentiment label from data

		sentiment = splitLine[1].strip()[-8:] # the last 8 characters are either positive or negative
		if sentiment == "positive":
			trueValues.append(1)
		elif sentiment == "negative":
			trueValues.append(0)
		else:
			print("Error: sentiment analysis not found at end of line!")
print(reviews[0])
print(trueValues)

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn 
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 

2. Create Features(X) table and Labels(y) array

In [ ]:
# X and y need to be np.arrays
X = np.zeros((len(reviews), len(sentimentWords)), dtype=bool) 	# Features
y = np.zeros(len(reviews), dtype=int) 		# Labels
posCount = 0
negCount = 0

for review in range(len(reviews)):
	for posWord in posWords:
		if posWord in reviews[review]: #make two loops, one for pos/neg
			X[review, posWords[posWord]] = True
			#print(f"Positive: {sentimentWords[word]}")
			posCount += 1 	
	for negWord in negWords:
		if negWord in reviews[review]:
			X[review, negWords[negWord]] = False
			#print(f"Negative: {sentimentWords[word]}")
			negCount += 1

	#print(posCount, negCount)
	if posCount >= negCount:
		y[review] = 1
	else:
		y[review] = 0
	posCount = 0
	negCount = 0

print(X.shape)
print(y.shape)
# for review in range(10):
# 	print(y[review], end=" ")
# 	print(X[review])
	#if y[review]:
		#print(f"Review {review} is positive!")


Custom Logistic Regression class

In [ ]:


class CustomLogisticRegression():
	# Should be 25-30 lines of code
	def __init__(self, learning_rate, num_iters):
		self.learning_rate = learning_rate
		self.num_iters = num_iters
		self.weights = None
		self.bias = None

	def fit(self, X, y):
		# Train the model using gradient descent
		# X is training features, y is labels

		# Sets the weights to the correct shape and initializes them to 0
		features = X.shape[1]
		self.weights = np.zeros(features) # weight for each feature
		self.bias = 0

		# Apply batch gradient descent on entire dataset
		# Gradient descent
		# This for loop was written by Claude 3.5 Sonnet and modified by myself
		for _ in range(self.num_iters):
			predictions = self.sigmoid(self.linearTransform(X)) # Calculate array of sigmoidal probabilities
			error = predictions - y # Calculate the difference between predicted and actual labels

			# Compute gradient for weights
			dw = (1 / len(y)) * np.dot(X.T, error) # X.T is transposed 
			# Compute gradient for bias
			db = (1 / len(y)) * np.sum(error) # Average of all errors

			# Update weights and biases
			self.weights -= self.learning_rate * dw
			self.bias -= self.learning_rate * db

	def sigmoid(self, z):
		#print("apply sigmoid to z")
		output = 1 / (1 + np.exp(-z)) # np.exp does e^(-z) for all samples in the reviews array
		#print("Sigmoid:", output)
		return output

	def predict(self, X): # Make predictions on test data by computing W DOT X + b and passing it through the sigmoid function, then converts to binary
		z = self.linearTransform(X) #might need to swap W X
		prob = self.sigmoid(z)
		prob = int(prob >= 0.5) # Convert to binary output
		return prob
	
	def linearTransform(self, X):
		z = np.dot(X, self.weights) + self.bias
		#print(f"Z: {z}")
		return z

scikit-learn https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.predict

In [ ]:
import sklearn.linear_model


for _ in range(5):
	# Shuffle input data
	# Split into 80% 20% split of training and test sets
	sklearn.model_selection.train_test_split

	skllr = sklearn.linear_model.LogisticRegression(solver='sag', C=0.001, max_iter=50).fit(X, y)
	mylr = CustomLogisticRegression(learning_rate=0.1, num_iters=50)

	predictionValues = skllr.predict(X[:2, :]) #X{array-like, sparse matrix} of shape (n_samples, n_features)
	print(predictionValues)
	print(skllr.score(X, y))
	
	print(skllr.confusion_matrix(trueValues, predictionValues))
	print(skllr.classification_report(trueValues, predictionValues, target_names=sentimentWords))

	print(mylr.confusion_matrix(trueValues, predictionValues))
	print(mylr.classification_report(trueValues, predictionValues, target_names=sentimentWords))

	# Calculate average confusion matrix
	#xTrain, xTest, yTrain, yTest = train_test_split(x, y, random_state=1)

	mylr.fit(X, y)